In [1]:
import bernielib as bl

In [2]:
import time

In [3]:
bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [4]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [5]:
ber.robotHome()

In [6]:
ber.pipetteHome()

# Function for beads amount calculations

In [7]:
# Those numbers are obtained 20041601, in Arnielib
def calcBeadsVol(sample_vol, DNA_size_cutoff):
    multiplier = 4.99325349e-01 - 9.91043764e+00 / DNA_size_cutoff + 2.57585836e+04 / DNA_size_cutoff ** 2
    return sample_vol * multiplier

In [8]:
calcBeadsVol(200, 100)

595.2158665200001

# Creating samples

In [9]:
def createReagentSample(type_name, sample_name, pos_col, pos_row, v):
    sample_type = bl.sample_type(type_name)
    sample = bl.sample(sample_name, sample_type)
    sample.place(ber.reagents_rack, pos_col, pos_row)
    sample.setVolume(v)
    return sample

In [10]:
def createSampleList(v_list, col):
    sample_type = bl.sample_type('eppendorf')
    sample_number = 0
    sample_list = []
    for v in v_list:
        sample = bl.sample('sample'+str(sample_number), sample_type)
        sample.place(ber.samples_rack, col, sample_number)
        sample.setVolume(v)
        sample_list.append(sample)
        sample_number += 1
    return sample_list

In [11]:
beads = createReagentSample('5ml', 'beads', 0, 0, 1000)
EtOH = createReagentSample('50ml', 'EtOH', 1, 0, 50000)
eluent = createReagentSample('50ml', 'eluent', 2, 0, 50000)
waste = createReagentSample('50ml', 'waste', 0, 1, 0)
water = createReagentSample('50ml', 'water', 1, 1, 45000)
EtOH80pct = createReagentSample('50ml', 'EtOH80pct', 2, 1, 50000)

In [12]:
samples_v_list = [100, 50, 200]

In [13]:
samples_cleanup_list = createSampleList([0, 0, 0], 1)

In [14]:
sampletest = samples_cleanup_list[0]

In [15]:
sampletest2 = samples_cleanup_list[1]

In [13]:
# Only use if the tubes were filled manually or if you are restarting
samples_cleanup_list = createSampleList(samples_v_list, 1)

# Pre-filling the samples with water to mock the conditions

In [22]:
ber.pickUpNextTip()

In [23]:
ber.move(z=0)

In [15]:
for sample, v in zip(samples_cleanup_list, samples_v_list):
    ber.transferLiquid(water, sample, v)

In [15]:
samples_cleanup_list[0].getVolume()

0

In [16]:
# Run this if the tip is already attached
ber.tip_attached=1

# Purification settings

In [16]:
upper_cutoff_list = [1000, 700, 300]
lower_cutoff_list = [300, 200, 100]

# Performing upper cutoff

In [18]:
# Upper cutoff beads volume calculation
upper_cut_v_list = []
for vol, size in zip(samples_v_list, upper_cutoff_list):
    v_beads = calcBeadsVol(vol, size)
    upper_cut_v_list.append(v_beads)

In [19]:
upper_cut_v_list

[51.517349495999994, 26.886805945102036, 150.49940826222223]

In [20]:
ber.moveMagnetsAway()

In [21]:
for v, sample in zip(upper_cut_v_list, samples_cleanup_list):
    print ("Transferring beads to sample ", sample.name, " of volume ", str(v))
    ber.transferLiquid(beads, sample, v)
    v_sample = sample.getVolume()
    v_updown = v_sample / 2
    if v_updown > 200:
        v_updown = 200
    ber.pipetteUpAndDown(sample, v_updown, 5)

Transferring beads to sample  sample0  of volume  51.517349495999994
Transferring beads to sample  sample1  of volume  26.886805945102036
Transferring beads to sample  sample2  of volume  150.49940826222223


In [24]:
830 // 200

4

In [25]:
830.1 // 200

4.0

In [26]:
51.517349495999994 // 200

0.0

In [21]:
ber.dumpTipToPosition(1, 3)

In [27]:
i=0

In [28]:
if i:
    print ('yes')
else:
    print ('no')

no


In [27]:
beads.purge()

In [17]:
ber.uptakeLiquid(beads, 200)

In [18]:
ber.dispenseLiquid(beads, 200, blow_extra=True)

In [17]:
ber.uptakeLiquid(beads, 200)

In [18]:
beads._isLowVolumeUptakeNeeded(205)

True

In [19]:
beads.getCloseToBottomVol()

91.85

In [20]:
beads._settingPresent('close_to_bottom_volume')

True

In [21]:
beads._getSetting('close_to_bottom_volume')

91.85

In [19]:
ber.pipetteUpAndDown(beads, 200, 3)

In [16]:
ber.uptakeLiquid(beads, 200, lag_vol=0)

In [18]:
ber.dispenseLiquid(beads, 200, plunger_retract=False, move_up_after=False, in_place=True)

In [21]:
ber.uptakeLiquid(beads, 100, lag_vol=0, in_place=True)

In [25]:
beads.calcAbsLiquidLevelFromVol(100, added_length=ber._calcExtraLength())

87.935

In [26]:
beads.calcNormalPipettingZ(100, 0, added_length=ber._calcExtraLength())

78.95

In [15]:
ber.pipetteUpAndDown(beads, 200, 3, dx=5, dy=-5)

In [16]:
ber.moveMagnetsAway()

In [15]:
ber.moveMagnetsTowardsTube()

In [18]:
realbeadslist = createSampleList([1000], 0)

In [19]:
realbeads = realbeadslist[0]

In [20]:
realbeads.getVolume()

1000

In [21]:
realbeads.data

{'rack_name': 'samples', 'rack_x_well': 0, 'rack_y_well': 0, 'volume': 1000}

In [29]:
ber.move(z=0)

In [25]:
ber.movePipetteToVolume(200)

In [20]:
ber.pipetteUpAndDown(realbeads, 200, 5, dx=4, v_in=100)

In [23]:
def mixBeads(robot, sample, insert_vol_list, radius, times=3):    
    
    for v_insert in insert_vol_list:
        robot.pipetteUpAndDown(sample, 200, times, dx=radius, v_in=v_insert)
        robot.pipetteUpAndDown(sample, 200, times, dx=radius, dy=radius, v_in=v_insert)
        robot.pipetteUpAndDown(sample, 200, times, dy=radius, v_in=v_insert)
        robot.pipetteUpAndDown(sample, 200, times, dx=-radius, dy=radius, v_in=v_insert)
        robot.pipetteUpAndDown(sample, 200, times, dx=-radius, v_in=v_insert)
        robot.pipetteUpAndDown(sample, 200, times, dx=-radius, dy=-radius, v_in=v_insert)
        robot.pipetteUpAndDown(sample, 200, times, dy=-radius, v_in=v_insert)
        robot.pipetteUpAndDown(sample, 200, times, dx=radius, dy=-radius, v_in=v_insert)

In [22]:
mixBeads(ber, realbeads, insert_vol_list=[100, 200], radius=3, times=3)

In [20]:
ber.pipetteUpAndDown(realbeads, 200, 3, dx=3, dy=3, v_in=100)

In [24]:
def mixInPlace(robot, v_uptake, v0, delay, times=3):
    for i in range(times):
        robot.movePipetteToVolume(v0)
        time.sleep(delay)
        robot.movePipetteToVolume(v_uptake)
        time.sleep(delay)

In [25]:
def mixBeads_1(robot, sample, v_in, radius, times=3):
    v0 = 0
    v_uptake = 200
    delay=0.5
    # Checking that sample has enough volume for mixing
    v_inside = sample.getVolume()
    if v_uptake > v_inside:
        v_uptake = v_inside
        
    z_immers_1 = sample.calcAbsLiquidLevelFromVol(v_in, added_length=robot._calcExtraLength())
    robot.movePipetteToVolume(v_uptake)
    robot.moveToSample(sample, z=z_immers_1)
    # 0 deg
    robot.moveAxisDelta('X', radius)
    mixInPlace(robot, v_uptake, v0, delay, times=times)
    # 45 deg
    robot.moveAxisDelta('Y', radius)
    mixInPlace(robot, v_uptake, v0, delay, times=times)
    # 90 deg
    robot.moveAxisDelta('X', -radius)
    mixInPlace(robot, v_uptake, v0, delay, times=times)
    # 135 deg
    robot.moveAxisDelta('X', -radius)
    mixInPlace(robot, v_uptake, v0, delay, times=times)
    # 180 deg
    robot.moveAxisDelta('Y', -radius)
    mixInPlace(robot, v_uptake, v0, delay, times=times)
    # 225 deg
    robot.moveAxisDelta('Y', -radius)
    mixInPlace(robot, v_uptake, v0, delay, times=times)
    # 270 deg
    robot.moveAxisDelta('X', radius)
    mixInPlace(robot, v_uptake, v0, delay, times=times)
    # 315 deg
    robot.moveAxisDelta('X', radius)
    mixInPlace(robot, v_uptake, v0, delay, times=times)
    
    v_max = sample.stype.getMaxVolume()
    z_out = sample.calcAbsLiquidLevelFromVol(v_max, added_length=robot._calcExtraLength())
    robot.moveToSample(sample, z=z_out)
    # Moving plunger all the way down
    robot.pipetteMove(40)
    robot.movePipetteToVolume(0)

In [26]:
mixBeads_1(ber, realbeads, 100, 3)

In [27]:
mixBeads_1(ber, realbeads, 200, 3)

In [28]:
mixBeads_1(ber, realbeads, 70, 0, times=1)

In [39]:
ber.transferLiquid(water, sampletest, 600)

In [30]:
ber.move(z=0)

In [47]:
ber.uptakeLiquid(realbeads, 100, v_insert_override=200)

In [48]:
ber.move(z=0)

In [49]:
ber.dispenseLiquid(sampletest, 100, blow_extra=True)

In [50]:
mixBeads_1(ber, sampletest, 100, 1, times=1)

In [54]:
mixBeads_1(ber, sampletest, 200, 1, times=1)

In [52]:
mixBeads_1(ber, sampletest, 300, 2, times=1)

In [53]:
mixBeads_1(ber, sampletest, 400, 3, times=1)

## mixing test 2

In [31]:
import re

In [32]:
re.split?

In [12]:
ber._getPipetteCurrentPosition_Raw()

-4.0

In [16]:
mix_scenario = {
    50:{
        0: {'X': 0, 'Y': 0},
    },
    100: {
        0: {'X': 2, 'Y': 0},
        1: {'X': 1.5, 'Y': 1.5},
        2: {'X': 0, 'Y': 2},
        3: {'X': -1.5, 'Y': 1.5},
        4: {'X': -2, 'Y': 0},
        5: {'X': -1.5, 'Y': -1.5},
        6: {'X': 0, 'Y': -2},
        7: {'X': 1.5, 'Y': -1.5},
        8: {'X': 0, 'Y': 0},
    },
    200: {
        0: {'X': 3, 'Y': 0},
        1: {'X': 2, 'Y': 2},
        2: {'X': 0, 'Y': 3},
        3: {'X': -2, 'Y': 2},
        4: {'X': -3, 'Y': 0},
        5: {'X': -2, 'Y': -2},
        6: {'X': 0, 'Y': -3},
        7: {'X': 2, 'Y': -2},
        8: {'X': 0, 'Y': 0},
    },
}

In [29]:
def mix_3(robot, sample, scenario, times=3):
    v_uptake = 200
    delay=0.5
    
    v_inside = sample.getVolume()
    if v_uptake > v_inside:
        v_uptake = v_inside
    
    robot.movePipetteToVolume(v_uptake)
    
    for v, points_dict in scenario.items():
        z_immers = sample.calcAbsLiquidLevelFromVol(v, added_length=robot._calcExtraLength())
        robot.moveToSample(sample, z=z_immers)
        
        for point_id, xy_rel_coord_dict in points_dict.items():
            dx = xy_rel_coord_dict['X']
            dy = xy_rel_coord_dict['Y']
            
            robot.moveAxisDelta('X', dx)
            robot.moveAxisDelta('Y', dy)
            
            robot._pipetteUpAndDownInPlace(delay=delay, times=times)
            
            robot.moveAxisDelta('X', -dx)
            robot.moveAxisDelta('Y', -dy)
            
    v_max = sample.stype.getMaxVolume()
    z_out = sample.calcAbsLiquidLevelFromVol(v_max, added_length=robot._calcExtraLength())
    robot.moveToSample(sample, z=z_out)

In [17]:
realbeadslist = createSampleList([1000], 0)

In [18]:
realbeads = realbeadslist[0]

In [30]:
mix_3(ber, realbeads, mix_scenario)

In [31]:
ber.move(z=0)

In [33]:
ber.dumpTipToWaste()

# Mixing 3

In [19]:
ber.pickUpNextTip()

In [20]:
ber.move(z=0)

In [22]:
realbeadslist = createSampleList([1000], 0)
realbeads = realbeadslist[0]

In [23]:
mix_scenario = {
    50:{
        0: {'X': 0, 'Y': 0},
    },
    100: {
        0: {'X': 2, 'Y': 0},
        1: {'X': 1.5, 'Y': 1.5},
        2: {'X': 0, 'Y': 2},
        3: {'X': -1.5, 'Y': 1.5},
        4: {'X': -2, 'Y': 0},
        5: {'X': -1.5, 'Y': -1.5},
        6: {'X': 0, 'Y': -2},
        7: {'X': 1.5, 'Y': -1.5},
        8: {'X': 0, 'Y': 0},
    },
    200: {
        0: {'X': 3, 'Y': 0},
        1: {'X': 2, 'Y': 2},
        2: {'X': 0, 'Y': 3},
        3: {'X': -2, 'Y': 2},
        4: {'X': -3, 'Y': 0},
        5: {'X': -2, 'Y': -2},
        6: {'X': 0, 'Y': -3},
        7: {'X': 2, 'Y': -2},
        8: {'X': 0, 'Y': 0},
    },
}

In [24]:
ber.mix(sample=realbeads, scenario=mix_scenario, times=1, mix_delay=0)

In [25]:
ber.move(z=0)

In [26]:
ber.dumpTipToWaste()